# Big Data HS 2025

## JSONiq tutorial - week 7

Every week, you will get a small tutorial notebook that introduces you to the JSONiq language with the RumbleDB engine. You can simply copy this notebook to the "notebooks" subfolder in your Exam MagicBox docker environment (the same environment that contains past exams, PostgreSQL, Spark, RumbleDB, etc).

The instructions are in week 1's tutorial.


Like last week, junst run the cell below to connect the Jupyter notebook with RumbleDB.

In [ ]:
%load_ext jsoniqmagic

## Navigating an existing JSON dataset

We continue with an existing dataset on the Web. Recall the following query, which opens the textual dataset as a sequence of strings.

In [ ]:
%%jsoniq
unparsed-text-lines("https://www.rumbledb.org/samples/hamlet.txt")

We are now going to logically simulate a MapReduce-like job that counts the number of occurences of each word.
First, we can add a for clause to iterate over the line (and do nothing else, so the query is equivalent to the previous one).

In [ ]:
%%jsoniq
for $line in unparsed-text-lines("https://www.rumbledb.org/samples/hamlet.txt")
return $line

Next, we can tokenize the lines. For simplicity, we will use spaces. The builtin tokenize() functions splits strings into several strings and by default, does this based on space characters.

Tokenizing each string in this way would correspond to the mapping phase of MapReduce (the value associated with each one of the words is implicitly 1).

In [ ]:
%%jsoniq
for $line in unparsed-text-lines("https://www.rumbledb.org/samples/hamlet.txt")
return tokenize($line)

We can also bind an intermediate variable to each token for convenience.

In [ ]:
%%jsoniq
for $line in unparsed-text-lines("https://www.rumbledb.org/samples/hamlet.txt")
for $token in tokenize($line)
return $token

We can make the intermediate key-value pairs explicit:

In [ ]:
%%jsoniq
for $line in unparsed-text-lines("https://www.rumbledb.org/samples/hamlet.txt")
for $token in tokenize($line)
let $pair := { $token : 1 }
return $pair

Next, we can use a group by clause, which essentially handles the shuffling and groups all words together that are the same.

After the group by clause, in each group, \\$t will be bound to the current token, and \\$pair (which precedes the group by) will now contain the *sequence* of all pairs with the current token as a key.

Thus, a JSONiq group by clause is similar to a SQL GROUP BY clause, but it is more generic because of its ability to bind each non-key variable to the sequence of all its values within a group, with no obligation to aggregate.

Note how we dynamically navigate to all the values in the sequence of pairs with \\$pair.\\$t, where \\$t is the current token and $pair contains all pairs with that token.

In [ ]:
%%jsoniq
for $line in unparsed-text-lines("https://www.rumbledb.org/samples/hamlet.txt")
for $token in tokenize($line)
let $pair := { $token : 1 }
group by $t := keys($pair)[1]
return 
{
    $t : sum($pair.$t)
}

We can clean up a bit by binding the count with an intermediate variable like so:

In [ ]:
%%jsoniq
for $line in unparsed-text-lines("https://www.rumbledb.org/samples/hamlet.txt")
for $token in tokenize($line)
let $pair := { $token : 1 }
group by $t := keys($pair)[1]
let $count := sum($pair.$t)
return 
{
    $t : $count
}

Which allows us to sort by descending counts and spot the most common tokens. The order by clause is similar to the SQL ORDER BY clause and also offers the choice between ascending and descending.

In [ ]:
%%jsoniq
for $line in unparsed-text-lines("https://www.rumbledb.org/samples/hamlet.txt")
for $token in tokenize($line)
let $pair := { $token : 1 }
group by $t := keys($pair)[1]
let $count := sum($pair.$t)
order by $count descending
return 
{
    $t : $count
}

Note that we can simplify the query a bit, but this is because JSONiq is more high-level than MapReduce and does not force the use of keys!

In [ ]:
%%jsoniq
for $line in unparsed-text-lines("https://www.rumbledb.org/samples/hamlet.txt")
for $token in tokenize($line)
group by $t := $token
order by count($token) descending
return 
{
    $t : count($token)
}

We can also limit the size of the output with a count clause. This would be similar to the use of LIMIT and OFFSET clauses in SQL, but the filtering can be done more generally than SQL with a where clause.

This is also an opportunity to say that the order of the clauses in JSONiq is very flexible and generic, whereas in SQL the clauses have to be in the order of SELECT FROM WHERE GROUP BY HAVING ORDER LIMIT OFFSet. In JSONiq, the only requirement is that the first clause is either a for or a let, and that the last clause is a return clause.

In [ ]:
%%jsoniq
for $line in unparsed-text-lines("https://www.rumbledb.org/samples/hamlet.txt")
for $token in tokenize($line)
group by $t := $token
order by count($token) descending
count $c
where $c le 10
return 
{
    $t : count($token)
}

# Connecting to PostgreSQL

It is also possible to use JSONiq to connect to PostgreSQL and other data sources. We just need to first make sure the jsoniq package is uptodate:

In [ ]:
%pip install jsoniq -U

Since we have a PostgreSQL instance running in the Big Data course's exam docker, we can very simply connect to it as follows. The first parameter is the connection string, which should not be changed unless you want to connect to a different server or database. The second one specifies the table to retrieve.

In [ ]:
%%jsoniq
postgresql-table("jdbc:postgresql://db:5432/postgres?user=postgres&password=example", "artists")

Since it is a table, we may as well switch the display to a pandas DataFrame with -pdf

In [ ]:
%%jsoniq -pdf
postgresql-table("jdbc:postgresql://db:5432/postgres?user=postgres&password=example", "artists")

FLWOR expressions, which were designed in the 2000s in the W3C XML Query Working Group by some of the same people who designed SQL, are considerably more flexible and powerful than SQL clauses because SQL clauses are restricted to a specific ordering (SELECT FROM WHERE GROUP BY HAVING ORDER BY LIMIT OFFSET) while FLWOR clauses are not. This is so useful, that even Google made the switch with its recently introduced [pipe syntax](https://cloud.google.com/blog/products/data-analytics/simplify-your-sql-with-pipe-syntax-in-bigquery-and-cloud-logging).

You can use FLWOR expressions to query PostgreSQL as follows. 

In [ ]:
%%jsoniq -pdf
let $connection := "jdbc:postgresql://db:5432/postgres?user=postgres&password=example"
return
for $artist in postgresql-table($connection, "artists")
for $released_by in postgresql-table($connection, "released_by")[$$.artist_id eq $artist.artist_id]
for $release in postgresql-table($connection, "releases")[$$.release_id eq $released_by.release_id]
let $id := $artist.artist_id, $name := $artist.name
group by $id, $name
let $num_releases := count($release)
order by $num_releases descending
return {
  "Artist id" : $id,
  "Artist name" : $name,
  "Number of releases" : $num_releases
} 

# Try your own queries!

This notebook is interactive. You can edit all queries above and also execute your own! We will show you more features every week.

In [ ]:
%%jsoniq
1+1

In [ ]:
%%jsoniq
1+1

In [ ]:
%%jsoniq
1+1

In [ ]:
%%jsoniq
1+1